# Be safe - who is in the biggest need for PPE?

In the time of Coronavirus outbreak, as a fresh graduate of Data Science, I would like to utilize my skills and help the World, as the World someday may help me. The goal of this notebook is to find by observation places and people that are in the biggest need of the Personal Protective Equipment (PPE) in the COVID-19 time.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

ecdc_infections = pd.read_csv('kaggle/input/uncover/UNCOVER/ECDC/current-data-on-the-geographic-distribution-of-covid-19-cases-worldwide.csv')
ecdc_infections.head()

FileNotFoundError: [Errno 2] File /kaggle/input/uncover/UNCOVER/ECDC/current-data-on-the-geographic-distribution-of-covid-19-cases-worldwide.csv does not exist: '/kaggle/input/uncover/UNCOVER/ECDC/current-data-on-the-geographic-distribution-of-covid-19-cases-worldwide.csv'

In [ ]:
ecdc_infections = pd.read_csv('/kaggle/input/uncover/UNCOVER/ECDC/current-data-on-the-geographic-distribution-of-covid-19-cases-worldwide.csv')
ecdc_infections.head()

In [ ]:
infections = pd.read_csv('/kaggle/input/uncover/UNCOVER/johns_hopkins_csse/johns-hopkins-covid-19-daily-dashboard-cases-by-country.csv')
infections.head()

In [ ]:
country_information = pd.read_csv('/kaggle/input/uncover/UNCOVER/HDE_update/inform-covid-indicators.csv')
country_information["population_living_in_urban_areas"] = country_information["population_living_in_urban_areas"].replace('No data', None)
country_information["population_living_in_urban_areas"] = country_information["population_living_in_urban_areas"].astype('float64')
country_information.head()

In [ ]:
tests = pd.read_csv('/kaggle/input/uncover/UNCOVER/HDE_update/HDE/total-covid-19-tests-performed-by-country.csv')
tests.head()

## Which country has the highest infection rate in comparison to the population?

Let's accumulate the active cases for all countries, then let's calculate the percentage of sick people with the density of this country. The data will be for the 5th of April, as I didn't find anything more accurate from UNCOVER dataset.

In [ ]:
def getDensity(countryName, country_info = country_information):
    code = country_info[country_info['country'] == countryName]['population_density'].values
    if len(code) == 0:
        return None
    return code[0]

def getPercentageOfUrbanArea(countryName, country_info = country_information):
    code = country_info[country_info['country'] == countryName]['population_living_in_urban_areas'].values
    if len(code) == 0:
        return None
    return code[0]

def getPopulation(countryName, country_info = ecdc_infections):
    pop = country_info[country_info['countriesandterritories'] == countryName]['popdata2018'].values
    if len(pop) == 0:
        return None
    return pop[0]

In [ ]:
country_data = infections[['country_region', 'last_update', 'confirmed', 'deaths','recovered','active']].copy()
country_data['density'] = country_data['country_region'].apply(lambda x: getDensity(x))
country_data['pop2018'] = country_data['country_region'].apply(lambda x: getPopulation(x))
country_data['perc_of_population_in_urban_area'] = country_data['country_region'].apply(lambda x: getPercentageOfUrbanArea(x))
country_data['perc_of_population_in_urban_area'] = np.around(country_data['perc_of_population_in_urban_area'].values, decimals=2)
country_data.head()

Now, as we have them all starting data in one data frame, let's show the percentage of active cases for countries.

In [ ]:
country_data['percentage'] = country_data['active']/country_data['pop2018']*100
plt.clf()
plt.figure(figsize=(15,10))
ax = sns.barplot(x="country_region", y="percentage", data=country_data.sort_values(by='percentage', ascending=False).head(10))
ax.set_title("Top 10 countries with the highest percentage of active cases in comparison to population")
plt.show()

The data is from the 5th of April, so it's before the major boom in the United States. We can observe the highest rate in the smallest countries. However see that in the plot we have 4th and 10th biggest European countries: Spain and Italy according to https://www.worldatlas.com/articles/the-largest-countries-in-europe.html.

Now let's see how the number of active cases compares to the population density.

In [ ]:
plt.clf()
plt.figure(figsize=(15,5))
ax = sns.barplot(x="density", y="percentage", data=country_data.sort_values(by="percentage", ascending=False).head(15))
ax.set_title("Top 15 highest percentage rate in countries compared to the density")
plt.show()

So we can tell that the higher infection rate is correlated with country density, we can observe more active cases above 0.1% of the population for countries with a density above 93 people per square kilometer. So the bigger the density, the higher probability that rate of infection will be high, of course, the density is calculated as the population over the country area - so in some cases, it's not true as we can see for the first country on the diagram. We can see, however, that in some cases, despite the high density the infection rate is relatively small (the last two bars), this can be caused by various reasons, which should be investigated (I'm counting on fast government reaction, people discipline and the fact that the epidemy may only be starting there).

Now let's see how the number of active cases looks like in countries based on the percentage of people living in urban areas.

In [ ]:
plt.clf()
plt.figure(figsize=(15,5))
ax = sns.barplot(x="perc_of_population_in_urban_area", y="percentage", data=country_data.sort_values(by="percentage", ascending=False).head(15))
ax.set_title("Top 15 countries with the highest percentage of active cases with the percentage of people living in urban areas.")
plt.show()

Based on this diagram we can tell that when more people are living in the urban areas the infection rate is higher, however, I would not put is as ground truth yet, as we don't know here what is the reason why some countries have this rate so low.

## Which US hospitals requires the equipment most?

In this section I would like to see how different states require equipment. Let's gather information about hospitals per state and number of cases per state.

In [ ]:
hospitals = pd.read_csv('/kaggle/input/uncover/UNCOVER/esri_covid-19/esri_covid-19/definitive-healthcare-usa-hospital-beds.csv')
hospitals['num_licens'] = hospitals['num_licens'].replace("****", 0)
hospitals['num_licens'] = hospitals['num_licens'].astype('int32')
hospitals['num_icu_be'] = hospitals['num_icu_be'].fillna(0)
hospitals['num_icu_be'] = hospitals['num_icu_be'].astype('int32')

covid_us = pd.read_csv('/kaggle/input/uncover/UNCOVER/covid_tracking_project/covid-statistics-by-us-states-totals.csv')
covid_us['hospitalized'] = covid_us['hospitalized'].fillna(0)
covid_us['hospitalized'] = covid_us['hospitalized'].astype('int32')

hospitals.head()

### Number of staff compared to number of cases

Let's see how many licensed personnel is available to treat the patients for different states.

In [ ]:
def getNumberOfStaff(state_code, hospitals = hospitals):
    h = hospitals.groupby('hq_state')['num_licens'].sum().reset_index()
    return h[h['hq_state'] == state_code]['num_licens'].values[0]

def getNumberOfIcuBeds(state_code, hospitals = hospitals):
    h = hospitals.groupby('hq_state')['num_icu_be'].sum().reset_index()
    return h[h['hq_state'] == state_code]['num_icu_be'].values[0]

In [ ]:
df_state = covid_us[['state', 'datemodified', 'hospitalized']].copy()
df_state['active'] = covid_us['positive'].fillna(0) - covid_us['death'].fillna(0)
df_state['active'] = df_state['active'].astype('int32')
df_state['licensed_staff'] = df_state['state'].apply(lambda state: getNumberOfStaff(state))
df_state['icu_beds'] = df_state['state'].apply(lambda state: getNumberOfIcuBeds(state))
df_state['active_per_icu_beds'] = df_state['active']/df_state['icu_beds']
df_state['hospitalized_per_icu_beds'] = (df_state['hospitalized']/df_state['icu_beds'])*100
df_state['active_per_staff'] = df_state['active']/df_state['licensed_staff']
df_state['hospitalized_per_staff'] = df_state['hospitalized']/df_state['licensed_staff']
# df_state.head()

The ICU (Intensive Care Unit) beds are a special type of bed for intensive care patients. They are crucial for treatment the most endangered patients. In the case of the COVID-19, they can support the most critically ill patients providing the respiration.

In [ ]:
plt.clf()
plt.figure(figsize=(25,10))
ax = sns.barplot(x='state', y='hospitalized_per_icu_beds', data=df_state.sort_values(by='hospitalized_per_icu_beds', ascending=False))
ax.set_title("Ratio of the hospitalized patients to the available ICU beds.")
ax.set_ylabel("Percentage of hospitalized patients to the number of all ICU beds in the state")
plt.show()

So New York has the highest rate of hospitalized people to the availability of ICU beds (over 400%) - due to a big number of hospitalized people the NY hospitals may require additional ICU as the state of some of them may get worse. According to https://newsroom.uw.edu/news/early-study-covid-19-patients-shows-high-mortality-rate the average time spent by COVID-19 patients is 10 days under ICU. We don't know however how many patients will require ICU. But from this diagram, we can tell that at least hospitals in the states: New York (NY), Louisiana(LA) and Connecticut (CT) will require additional ICUs in a short period of time.

Now let's see how the number looks for the staff for hospitalized and non-hospitalized patients.

In [ ]:
plt.clf()
plt.figure(figsize=(25,10))
ax = sns.barplot(x='state', y='hospitalized_per_staff', data=df_state.sort_values(by='hospitalized_per_staff', ascending=False))
ax.set_title("Ratio of the hospitalized patients to the licensed hospital staff.")
ax.set_ylabel("Ratio")
plt.show()

The NY staff have the highest number of hospitalized patients per one employee - 1 person handles around 4 patients and we here are only the hospitalized patients - the statistics are not showing the information how many of the staff is not working at the hospital directly - e.g. there is no information about the paramedics which are transporting patients between hospitals. If the number of hospitalized patients will increase the hospitals apart of the protective equipment will require staff to perform exchange of tired personnel - the most endangered hospitals are in the New York (NY), Connecticut (CT), Guam (GU) and Louisiana (LA).

Now to see how the situation can change let's see the ratio of the active cases to the staff available.

In [ ]:
plt.clf()
plt.figure(figsize=(25,10))
ax = sns.barplot(x='state', y='active_per_staff', data=df_state.sort_values(by='active_per_staff', ascending=False))
ax.set_title("Ratio of the hospitalized patients to the licensed hospital staff.")
ax.set_ylabel("Ratio")
plt.show()

Again the situation in New York may change and the staff will require additional support (the number of active cases exceeds the number of the staff available). We need to keep in mind that for in this case not only the hospital staff requires the protective equipment, but also other people e.g. police and people that have to maintain the economy like shopkeepers or even cleaning staff, to maintain their safety and the safety of the environment. In the highest need, there are again New York, but also New Jersey, Washington, Connecticut, and Massachusetts.